In [3]:
import glob


In [4]:
!ls

controller_exploration.ipynb  duckie_to_coco_bezier.ipynb  output
detectron2_duckie.ipynb       duckie_to_coco.ipynb	   __pycache__
duckie_real_train.json	      input.jpg			   README.md
duckie_real_val.json	      merge_coco_datasets.ipynb    utils.py
duckie_sim_val2.json	      merged_dataset


In [5]:
json_files = glob.glob("../solution/exercise_ws/datalog/*.json")

In [6]:
import json
with open(json_files[0]) as f:
    data = json.load(f)
    

In [7]:
data

{'white': [[[0.5684783137644999, -0.06138166862588269],
   [0.5231590882309906, -0.2866465247018372]]],
 'yellow': [[[0.1944266551323723, 0.19022132370630465],
   [0.1909799123421903, 0.1893387682271845]],
  [[0.19147555290789747, 0.20556691494868395],
   [0.18492557419408556, 0.2032578468105979]],
  [[0.18510143102153054, 0.2095445954053378],
   [0.18200459053965143, 0.20833277294584732]],
  [[0.18208735300978754, 0.21142548808606634],
   [0.17902498024379318, 0.20716682374756676]],
  [[0.1785631638855626, 0.18912224615684894],
   [0.18159379493706146, 0.19298192880627535]],
  [[0.184663422807754, 0.1938861329933145],
   [0.18423080713617926, 0.17842044988706304]],
  [[0.19068557542487222, 0.1797016581714802],
   [0.19078343605878048, 0.18290578800910798]],
  [[0.22014533522033608, 0.1262900292404056],
   [0.21135218013892507, 0.12720914465879637]],
  [[0.2114905081945375, 0.13069807692672566],
   [0.20737516132466297, 0.1310511013399357]],
  [[0.20816013544168907, 0.1517290329785236]

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
    
from scipy.optimize import curve_fit
import numpy as np

def get_xy(points):
    x = []
    y = []
    for point in points:
        x.append(point[0][0])
        x.append(point[1][0])
        y.append(point[0][1])
        y.append(point[1][1])
    return (x,y)

def fit_and_show(x,y, c="b"):
    a,b = fit(x,y)
    show_line(a,b,c)
    return a,b


def fit(x,y, c="b"):
    if len(x)>3:
        popt, pcov = curve_fit(line_func, x, y)
        return popt
    else:
        raise ValueError("Not enough data points")

        
def line_func(x, a, b):
    #return a * np.exp(-b * x) + c
    #return a * x**2 + b*x + c
    return a*x + b

def show_line(a,b,c):
    model_x = np.arange(0,1,0.01)
    model_y = line_func(model_x, a,b)
    plt.plot(model_x, model_y, c)

def get_aim_point(a,b, dist, offset):
    x, y = dist,dist*a+b-offset
    return (x,y)
    
last_aim_point = (0,0.2)

def show_json(filename, offset=None, dist=None):
    global last_aim_point
    if offset:
        offset = offset
    else:
        offset = 0.12
    if dist:
        dist=dist
    else:
        dist = 0.2
    with open(filename) as f:
        data = json.load(f)
    
    x, y = get_xy(data["yellow"])
    plt.scatter(x,y, c="y")
    yellow_aim_point = None
    white_aim_point = None
    try:
        a,b = fit_and_show(x,y, "y")
    
        yellow_aim_point = get_aim_point(a,b,dist,offset)
        plt.scatter(*yellow_aim_point, marker="X", c="y")
    except ValueError:
        pass
    
    try:
        x, y = get_xy(data["white"])
        x_right=[]
        y_right=[]
        if yellow_aim_point:
            print("bli")
            print(a,b)
            for xval,yval in zip(x,y):
                if yval < float(a)*xval+float(b):
                    x_right.append(xval)
                    y_right.append(yval)
        else:
            x_right = x
            y_right = y

        plt.scatter(x_right,y_right, c="k")
        a,b = fit_and_show(x_right,y_right, "k")
    
        plt.scatter(0,0, marker="D")
    
        white_aim_point = get_aim_point(a,b,dist,-offset)
        plt.scatter(*white_aim_point, marker="X", c="k")
    except ValueError:
        pass
    plt.xlim([0,1])
    plt.ylim([-1,1])
    
    aim_point=None
    if yellow_aim_point:
        aim_point = yellow_aim_point
        if white_aim_point:
            aim_point = (
                            ((yellow_aim_point[0] + white_aim_point[0]) / 2),
                            ((yellow_aim_point[1] + white_aim_point[1]) / 2)
            )
    else:
        aim_point = white_aim_point
    
    if aim_point is None:
        aim_point = last_aim_point
    else:
        last_aim_point=aim_point
            
    plt.scatter(*aim_point, marker="X", c="r")
    
    print(yellow_aim_point, white_aim_point, aim_point)
    
    

In [62]:
#a,b = fit_and_show(x,y, "y")
#x, y = get_xy(data["white"])

In [68]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
i=0
@interact(x=sorted(json_files),offset=0.18, dist=0.2)
def g(x, offset, dist):
    global i
    show_json(x, offset, dist)
    i=i+1
    plt.savefig(f"animation/img_{i:03d}.png")

interactive(children=(Dropdown(description='x', options=('../solution/exercise_ws/datalog/segments_101.json', …